In [1]:
import pandas as pd 
import sys
import os
import openai
import numpy as np
from tqdm import tqdm
import json

In [2]:
hash_input = pd.read_csv('hashtag_top300_clean_20250122.csv')
hash_input.head()

,hashtags
0,iheartawards
1,bestfanarmy
2,bbb22
3,louies
4,fifaworldcup


In [3]:
def get_tags(df, num, column='hashtags'):
    input_size = len(hash_input)
    curr =  0
    tags_list = []
    while (curr<=input_size-num):
        tags_list.append(list(df.iloc[curr:curr+num][column]))
        curr += num
    tags_list.append(list(df.iloc[curr:input_size][column]))
    tags_list = [tags for tags in tags_list if tags != []]
    return tags_list

In [4]:
tags_list = get_tags(hash_input, 50)

In [5]:
def get_classification(prompt, sys_prompt, client, model):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": sys_prompt},
                {"role": "user", "content": prompt}
            ],
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"

In [6]:
sys_prompt = 'You are to help me decide whether whether hashtags are likely politically charged. You will be given a list of hashtags. please help determine whether a hashtag is likely politically charged or not. please do not use code, but instead use discretionary knowledge to evaluate them one by one. response in the format of a dictionary, in which the keys are the original hashtags and values are binary numbers, 0 represent not likely and 1 represent likely. the keys should be wrapped in double quotes. Please give plain text instead of any other format, with no space or special characters between paris. Do not add any additional content.'
client = openai.OpenAI(os.getenv("OPENAI_API_KEY"))
model = "gpt-4o"

In [7]:
classification_matches = []
for tags in tqdm(tags_list): 
    tags_str = str(tags)
    prompt = tags_str
    #prompt = tags_str + f'length of list: {len(tags)}'
    classification = get_classification(prompt, sys_prompt, client, model).strip()
    classification = classification + '}' if classification[-1]!='}' else classification
    try: 
        my_dict = json.loads(classification)
        classification_matches.append(my_dict)
    except Exception as e:
        print("An error occurred:", e)
    

100%|██████████| 122/122 [16:11<00:00,  7.96s/it]


In [8]:
combined_classification = {}
for d in classification_matches:
    combined_classification.update(d)
classification_df = pd.DataFrame(list(combined_classification.items()), columns=['hashtags', 'likely_politically_charged'])
classification_df

,hashtags,likely_politically_charged
0,iheartawards,0
1,bestfanarmy,0
2,bbb22,0
3,louies,0
4,fifaworldcup,0
...,...,...
6079,moongirlanddevildinosaur,0
6080,bidenharris2024,1
6081,arrestkatiehobbs,1
6082,nikkihaley,1


In [10]:
hashtag_with_classification = pd.merge(hash_input, classification_df, on="hashtags", how="left")
hashtag_with_classification

,hashtags,likely_politically_charged
0,iheartawards,0
1,bestfanarmy,0
2,bbb22,0
3,louies,0
4,fifaworldcup,0
...,...,...
6079,moongirlanddevildinosaur,0
6080,bidenharris2024,1
6081,arrestkatiehobbs,1
6082,nikkihaley,1


In [200]:
set(hash_input['hashtags']) - set(classification_df['hashtags'])

set()